##### This notebook demonstrates a demo of how churn-prediction model can be trained and deployed on truefoundry platform.

## Try this Notebook in Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truefoundry/truefoundry-examples/blob/main/end-to-end-examples/churn-prediction/truefoundry-churn-prediction.ipynb)

# 🛠 Setup

In [ ]:
pip install -U servicefoundry mlfoundry --quiet

In [ ]:
import logging
[logging.root.removeHandler(h) for h in logging.root.handlers]
logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(name)s] %(levelname)-8s %(message)s')

In [ ]:
from getpass import getpass
import os

WORKSPACE = input("Enter Your Workspace FQN here: ")

os.environ["TFY_HOST"] = "<Enter the url of your Truefoundry Dashboard here>"
os.environ["TFY_API_KEY"] = getpass("Please enter your API Key: ")

In [ ]:
!git clone https://github.com/truefoundry/truefoundry-examples.git
%cd truefoundry-examples
%cd end-to-end-examples/churn-prediction

# ⚡ Deploying a training Job - Quick Start
Here we will create a training job that will run on truefoundry's infrastructure and save the results.

### Training Script


In [43]:
%cd train

/home/jovyan/truefoundry-examples/end-to-end-examples/churn-prediction/train


In [44]:
!pygmentize main.py

import pandas as pd
from sklearn.neighbors import KNeighborsClassifier as Classification
import mlfoundry as mlf
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

def experiment_track(model, params, metrics, X_train, X_test):
    # initialize the mlfoundry client.
    mlf_api = mlf.get_client()

    # create a ml repo
    mlf_api.create_ml_repo("churn-pred")
    # create a run
    mlf_run = mlf_api.create_run(
        ml_repo="churn-pred", run_name="churn-train-job"
    )
    # log the hyperparameters
    mlf_run.log_params(params)
    # log the metrics
    mlf_run.log_metrics(metrics)
    # log the model
    model_version = mlf_run.log_model(
        name="churn-model",
        model=model,
        # specify the framework used (in this case sklearn)
        framework=mlf.ModelFramework.SKLEARN,
        description="churn-pred

### Defining the requirements

In [45]:
!pygmentize requirements.txt

matplotlib==3.1.3
matplotlib-inline==0.1.6
mlfoundry>=0.7.2,<0.8.0
pandas==1.5.0


### Deploy as a training job

In [ ]:
import argparse
import logging
import os
from servicefoundry import Build, PythonBuild, Resources, Job, Param, LocalSource

logging.basicConfig(level=logging.INFO)

# Defining the job
job = Job(
    name="churn-prediction-train",
    image=Build(
        build_spec=PythonBuild(
            command="python main.py --n_neighbors {{n_neighbors}} --weights {{weights}} --algorithm {{algorithm}} --power {{power}} --leaf_size {{leaf_size}} --metric {{metric}} --n_jobs {{n_jobs}}",
            requirements_path="requirements.txt",
        ) ,
        build_source=LocalSource(local_build=False)
    ),
    params=[
        Param(name="n_neighbors", default='5', description="Number of neighbors to use by default"),
        Param(name="weights", default='uniform', description="Weight function used in prediction.  Possible values: uniform, distance"),
        Param(name="algorithm", default='auto', description="Algorithm used to compute the nearest neighbors: possible values: 'auto', 'ball_tree', 'kd_tree', 'brute'"),
        Param(name="power", default='2', description="Power parameter for the Minkowski metric. When p = 1, this is manhattan_dist, and euclidean_dist p = 2"),
        Param(name="leaf_size", default='30', description="Leaf size passed to BallTree or KDTree"),
        Param(name="metric", default='minkowski', description="The distance metric to use for the tree. The default metric is minkowski,"),
        Param(name="n_jobs", default='1', description="The number of parallel jobs to run for neighbors search"),
    ],
    resources=Resources(
        cpu_request=1, cpu_limit=1, memory_request=2000, memory_limit=2000,
    ),
)

# Finally, we call deploy to push it to Truefoundry platform
deployment = job.deploy(workspace_fqn=WORKSPACE)

# Trigger the training runs
To trigger the job from UI:
Click on https://app.truefoundry.com/deployments?tab=jobs and trigger a run.

For triggering from python-sdk, run the following snippet.

In [ ]:
from servicefoundry import trigger_job

application_fqn = "ctl-demo:demo:churn-prediction-train"
trigger_job(
    application_fqn=application_fqn,
    params={
        "n_neighbors": "4"
    }
)

## Train Multiple Models with different hyperparameters

In [ ]:
from itertools import product

# modeify to dict
hyperparameters = {
    'n_neighbors': ["5", "10", "15"],
    'weights': ['uniform', 'distance'],
    'power': ['1', '2'],
    'leaf_size': ['30', '40'],
    'metric': ['minkowski', 'euclidean'],
}

# generate all possible combinations using itertools
values = list(product(*hyperparameters.values()))

for value in values:
    # convert to dict
    params = dict(zip(hyperparameters.keys(), value))
    trigger_job(
        application_fqn=application_fqn,
        params=params
    )

# ☁ Deploying the trained Model
Here we will deploy the model saved in truefoundry's model registry at the time of model training.

In [ ]:
MODEL_VERSION_FQN = input("Please enter the model version fqn:-")

In [ ]:
import logging

from servicefoundry import ModelDeployment, Resources, TruefoundryModelRegistry, Endpoint

logging.basicConfig(level=logging.INFO)

model_deployment = ModelDeployment(
    name=f"churn-prediction",
    model_source=TruefoundryModelRegistry(model_version_fqn=MODEL_VERSION_FQN),
    resources=Resources(cpu_request=0.2, cpu_limit=0.5, memory_request=500, memory_limit=1000),
    endpoint=Endpoint(host="churn-prediction-model-demo.demo2.truefoundry.tech")
)

model_deployment.deploy(workspace_fqn=WORKSPACE)

## Lets try out the deployed model

In [29]:
model_deployed_url = input("Enter the deployed url for the model")

Enter the deployed url for the model https://churn-prediction-model-demo.demo2.truefoundry.tech


In [21]:
import requests

def _predict(input_dict, model_deployed_url):
    json_body = {
        "parameters": {
            "content_type": "pd"
        },
        "inputs": []
    }

    for k, v in input_dict.items():
        json_body["inputs"].append(
            {
                "name": k,
                "datatype": "FP32",
                "data": [v],
                "shape": [1]
            }
        )
    response = requests.post(
        url=f"{model_deployed_url}/v2/models/churn-model/infer",
        json=json_body
    )
    return response.json()["outputs"][0]["data"][0]


In [ ]:
input_dict = {
    "CreditScore": 700,
    "Age": 42,
    "Tenure": 2,
    "Balance": 1000000,
    "NumOfProducts": 1,
    "HasCrCard": 1,
    "IsActiveMember": 0,
    "EstimatedSalary": 10948.88
}

output = _predict(input_dict, model_deployed_url)

print(output)

#  ☁  Deploying  Model's Demo as a Service
In this section we will deploy a training job that trains a SVM model on iris dataset

In [22]:
%cd ../demo

/home/jovyan/truefoundry-examples/end-to-end-examples/churn-prediction/demo


### Creating a gradio demo

In [24]:
!pygmentize demo.py

import gradio as gr
import datetime
import os
import requests
from urllib.parse import urljoin

# Endpoint of the deployed model
MODEL_DEPLOYED_URL = os.environ['MODEL_DEPLOYED_URL']

# specifying the desired input components
inputs = [
    gr.Number(label="CreditScore", value=619),
    gr.Number(label="Age", value=42),
    gr.Number(label="Tenure", value=2),
    gr.Number(label="Balance", value=0),
    gr.Number(label="NumOfProducts", value=1),
    gr.Number(label="HasCrCard", value=1),
    gr.Number(label="IsActiveMember", value=1),
    gr.Number(label="EstimatedSalary", value=101348.88)
]

def predict(*val):
    # request body in dictionary format
    json_body = {"parameters": {
        "content_type": "pd"
    }, "inputs": []}

    # add the values into inputs list of json_body
    for v, inp in zip(val, inputs):
        json_body["inputs"].append(
            {
                "name": inp.label,
                "datatype": "FP32",
                "data": [v],
                "sha

### Defining the requirements

In [25]:
!pygmentize requirements.txt

requests==2.28.1
gradio==3.17.1
urllib3==1.26.12


### Deploying the gradio demo as a 'Service'

In [30]:
MODEL_DEPLOYED_URL = input("Please enter the deployed url for model (found on dashboard)")

Please enter the deployed url for model (found on dashboard) https://churn-prediction-model-demo.demo2.truefoundry.tech


In [ ]:
import logging

from servicefoundry import Build, PythonBuild, Resources, Service

logging.basicConfig(level=logging.INFO)

# creating a service object and defining all the configurations
service = Service(
    name="churn-prediction-demo",
    image=Build(
        build_spec=PythonBuild(
            command="python demo.py",
            python_version="3.9",
            requirements_path="requirements.txt"
        ),
    ),
    env={
        "MODEL_DEPLOYED_URL": MODEL_DEPLOYED_URL,
    },
    ports=[{"port": 8080, "host": "churn-pred-demo.demo2.truefoundry.tech"}], #In public cloud deployment TrueFoundry exposes port 8080
    resources=Resources(
        cpu_request=0.5, cpu_limit=0.5, memory_limit=2500, memory_request=1500
    ),
    replicas=1
)
service.deploy(workspace_fqn=WORKSPACE)
